In [123]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point, Polygon, MultiPolygon
import psycopg2
import psycopg2.extras
import json
import numpy as np
from scipy.stats import zscore
import geopandas as gpd
import matplotlib.pyplot as plt

In [124]:
incidents = pd.read_csv(os.getcwd()+'/data/incidents/excel/incidentsRaw.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [188]:
businesses = pd.read_csv(os.getcwd()+'/data/Businesses.csv')
income = pd.read_csv(os.getcwd()+'/data/Income.csv')
polling = pd.read_csv(os.getcwd()+'/data/PollingPlaces2019.csv')
population = pd.read_csv(os.getcwd()+'/data/Population.csv')
stops = pd.read_csv(os.getcwd()+'/data/Stops.txt')

In [126]:
catch_future = gpd.read_file(os.getcwd()+'/data/catchments/catchments_future.shp')
catch_primary = gpd.read_file(os.getcwd()+'/data/catchments/catchments_primary.shp')
catch_secondary = gpd.read_file(os.getcwd()+'/data/catchments/catchments_secondary.shp')

## Database Ingestion 

In [127]:
credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn
    
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

Checking for a successful connection 

In [128]:
db, conn = pgconnect(credentials)

Connected successfully.


In [60]:
# This is creating errors??? DO NOT RUN 
#conn.execute(text("""
#create schema if not exists SydneyGeo;
#set search_path to SydneyGeo;
#"""))

In [129]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [193]:
conn.execute("""
DROP TABLE IF EXISTS gdf;
CREATE TABLE gdf (
    sa2_code21 VARCHAR(500),
    sa2_name21 VARCHAR(255),
    chg_flag21 INT,
    chg_lbl21 VARCHAR(50),
    sa3_code21 VARCHAR(500),
    sa3_name21 VARCHAR(500),
    sa4_code21 VARCHAR(500),
    sa4_name21 VARCHAR(500),
    gcc_code21 VARCHAR(500),
    gcc_name21 VARCHAR(500),
    ste_code21 VARCHAR(500),
    ste_name21 VARCHAR(500),
    aus_code21 VARCHAR(500),
    aus_name21 VARCHAR(500),
    areasqkm21 FLOAT,
    loci_uri21 VARCHAR(255),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);"""
)

In [194]:
gdf = gpd.read_file(os.getcwd()+'/data/sa2/SA2_2021_AUST_GDA2020.shp')
gdf.columns = map(str.lower, gdf.columns)

In [195]:
srid = 4283
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

gdf_old = gdf.copy()  
gdf = gdf.dropna(subset=['geometry'])
#gdf = gdf_clean[~gdf_clean['geometry'].is_empty]
gdf['geom'] = gdf['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
gdf = gdf.drop(columns="geometry")

In [196]:
gdf = gdf[gdf['ste_name21'] == 'New South Wales']
gdf = gdf[gdf['gcc_name21'] == 'Greater Sydney']

In [197]:
gdf

,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,gcc_code21,gcc_name21,ste_code21,ste_name21,aus_code21,aus_name21,areasqkm21,loci_uri21,geom
28,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.413733024921 -33.465580583...
29,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.37484081570685 -33.5005199...
30,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.20449037540152 -33.5328022...
31,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.37193611462118 -33.4369790...
32,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.32348639265098 -33.4277852...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.07362997413264 -34.0563789...
638,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.05006441218998 -34.0215774...
639,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((150.99568346574816 -34.0536082...
640,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.03954821100714 -34.0417452...


In [198]:
gdf.to_sql("gdf", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from gdf")

,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,gcc_code21,gcc_name21,ste_code21,ste_name21,aus_code21,aus_name21,areasqkm21,loci_uri21,geom
0,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000010...
2,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000200000001030000000100000085...
3,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000041...
4,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB100000010000000103000000010000007E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000046...
369,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000089...
370,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB100000010000000103000000010000008E...
371,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB10000001000000010300000001000000A1...


Now loading the stops data set

In [137]:
stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code
0,200039,200039.0,"Central Station, Eddy Av, Stand A",-33.882206,151.206665,NaN,200060,0,NaN
1,200054,200054.0,"Central Station, Eddy Av, Stand D",-33.882042,151.206991,NaN,200060,0,NaN
2,200060,NaN,Central Station,-33.884084,151.206292,1.0,NaN,0,NaN
3,201510,NaN,Redfern Station,-33.891690,151.198866,1.0,NaN,0,NaN
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",-33.893329,151.198882,NaN,201510,0,NaN
...,...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",-33.822016,151.078797,NaN,21271,1,B
114714,2137185,2137185.0,"Cabarita Wharf, Side A",-33.840669,151.116926,NaN,21371,1,1A
114715,2137186,2137186.0,"Cabarita Wharf, Side B",-33.840769,151.116899,NaN,21371,1,1B
114716,21501,21501.0,Parramatta Wharf,-33.813904,151.010577,NaN,2150112,1,NaN


In [138]:
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops = stops.drop(columns=['stop_lat', 'stop_lon']) 
stops

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,NaN,POINT (151.20666 -33.88221)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,NaN,POINT (151.20699 -33.88204)
2,200060,NaN,Central Station,1.0,NaN,0,NaN,POINT (151.20629 -33.88408)
3,201510,NaN,Redfern Station,1.0,NaN,0,NaN,POINT (151.19887 -33.89169)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,NaN,POINT (151.19888 -33.89333)
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",NaN,21271,1,B,POINT (151.07880 -33.82202)
114714,2137185,2137185.0,"Cabarita Wharf, Side A",NaN,21371,1,1A,POINT (151.11693 -33.84067)
114715,2137186,2137186.0,"Cabarita Wharf, Side B",NaN,21371,1,1B,POINT (151.11690 -33.84077)
114716,21501,21501.0,Parramatta Wharf,NaN,2150112,1,NaN,POINT (151.01058 -33.81390)


In [139]:
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [140]:
conn.execute("""
DROP TABLE IF EXISTS Stops;
CREATE TABLE Stops (
    stop_id VARCHAR(10),
    stop_code FLOAT,
    stop_name VARCHAR(255),
    location_type FLOAT,
    parent_station VARCHAR(255),
    wheelchair_boarding INT,
    platform_code VARCHAR(255),
    geom GEOMETRY(POINT,4283)
);"""
)

In [141]:
stops.to_sql("stops", con=conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})

In [142]:
query(conn, "select * from stops")

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,None,0101000020BB100000FFA631FF9CE66240A1FF6524ECF0...
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,None,0101000020BB1000002F928BAC9FE66240E33DC7C1E6F0...
2,200060,NaN,Central Station,1.0,None,0,None,0101000020BB100000817FA2F299E662408FF33DAC29F1...
3,201510,NaN,Redfern Station,1.0,None,0,None,0101000020BB1000009E57611C5DE6624060304CE622F2...
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,None,0101000020BB100000DBF9333D5DE662403DFA6B9D58F2...
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",NaN,21271,1,B,0101000020BB100000AF9B3D8185E262408F52D7D537E9...
114714,2137185,2137185.0,"Cabarita Wharf, Side A",NaN,21371,1,1A,0101000020BB100000EB409ADCBDE3624089CE4C0B9BEB...
114715,2137186,2137186.0,"Cabarita Wharf, Side B",NaN,21371,1,1B,0101000020BB100000C4F9BEA2BDE362403EB375529EEB...
114716,21501,21501.0,Parramatta Wharf,NaN,2150112,1,None,0101000020BB100000E443E4A456E0624025C1A4032EE8...


Now executing a query to get placement of stops in SA2 areas

In [143]:
sql = """
SELECT gf.sa2_code21, gf.sa2_name21, COUNT(s.stop_id) AS stop_count
FROM gdf gf LEFT JOIN Stops s ON ST_Contains(gf.geom, s.geom)
GROUP BY gf.sa2_code21, gf.sa2_name21
ORDER BY stop_count DESC;
"""
public_transport_stops = query(conn, sql)

Working with School Data

In [148]:
schools = pd.concat([catch_primary, catch_secondary, catch_future])
schools.columns = map(str.lower, schools.columns)

In [149]:
srid = 4283
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

schoolsold = schools.copy()  
schools_clean = schools.dropna(subset=['geometry'])
schools = schools_clean[~schools_clean['geometry'].is_empty]
schools['geom'] = schools['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
schools = schools.drop(columns="geometry")
schools

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geom
0,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.1833640465581 -33.74748398...
1,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.0451821055135 -33.77303212...
2,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((150.92567327976582 -33.7296030...
3,3918,PRIMARY,Killara PS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.1537883781186 -33.75586174...
4,3396,PRIMARY,Waterfall PS,None,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((150.94786340261896 -34.1059463...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,8535,HIGH_COED,Concord HS,20240207,0,0,0,0,0,0,0,2025,2025,2025,2025,2025,2025,NaN,MULTIPOLYGON (((151.1055060898682 -33.83344860...
29,8596,HIGH_COED,Wollumbin HS,20240116,0,0,0,0,0,0,0,2025,2025,2025,2025,2025,2025,NaN,MULTIPOLYGON (((153.22573104983442 -28.3714522...
30,8129,HIGH_COED,Murwillumbah HS,20240116,0,0,0,0,0,0,0,2025,2025,2025,2025,2025,2025,NaN,MULTIPOLYGON (((153.3800034197263 -28.34046282...
31,8922,HIGH_COED,Jerrabomberra HS,20220615,0,0,0,0,0,0,0,2025,2025,2025,2025,2026,2027,NaN,MULTIPOLYGON (((149.20599879911867 -35.3710405...


In [150]:
conn.execute("""
DROP TABLE IF EXISTS Schools;
CREATE TABLE Schools (
    USE_ID INT,
    CATCH_TYPE VARCHAR(255),
    USE_DESC VARCHAR(255),
    ADD_DATE DATE,
    KINDERGART CHAR(4),
    YEAR1 CHAR(4),
    YEAR2 CHAR(4),
    YEAR3 CHAR(4),
    YEAR4 CHAR(4),
    YEAR5 CHAR(4),
    YEAR6 CHAR(4),
    YEAR7 CHAR(4),
    YEAR8 CHAR(4),
    YEAR9 CHAR(4),
    YEAR10 CHAR(4),
    YEAR11 CHAR(4),
    YEAR12 CHAR(4),
    PRIORITY VARCHAR(50),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);"""
) 

In [151]:
schools.to_sql("schools", con=conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from schools")

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geom
0,2404,PRIMARY,Lindfield EPS,2021-12-19,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB10000001000000010300000001000000BE...
1,4393,PRIMARY,Carlingford WPS,2022-02-23,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB1000000100000001030000000100000065...
2,4615,PRIMARY,Caddies Ck PS,2018-12-10,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB1000000100000001030000000100000056...
3,3918,PRIMARY,Killara PS,2021-12-19,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB1000000100000001030000000100000042...
4,3396,PRIMARY,Waterfall PS,None,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB10000001000000010300000001000000E1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,8535,HIGH_COED,Concord HS,2024-02-07,0,0,0,0,0,0,0,2025,2025,2025,2025,2025,2025,None,0106000020BB100000010000000103000000010000003E...
2130,8596,HIGH_COED,Wollumbin HS,2024-01-16,0,0,0,0,0,0,0,2025,2025,2025,2025,2025,2025,None,0106000020BB100000010000000103000000010000000C...
2131,8129,HIGH_COED,Murwillumbah HS,2024-01-16,0,0,0,0,0,0,0,2025,2025,2025,2025,2025,2025,None,0106000020BB10000001000000010300000001000000AF...
2132,8922,HIGH_COED,Jerrabomberra HS,2022-06-15,0,0,0,0,0,0,0,2025,2025,2025,2025,2026,2027,None,0106000020BB1000000100000001030000000100000039...


In [152]:
#WEIRD QUESTION: why does ST_Intersects work but not Contains or Within?

sql = """
SELECT gf.sa2_code21, gf.sa2_name21, COUNT(s.use_id) AS school_count
FROM gdf gf LEFT JOIN schools s ON ST_Intersects(gf.geom, s.geom)
GROUP BY gf.sa2_code21, gf.sa2_name21
ORDER BY school_count DESC;
"""
sa2_school_count = query(conn, sql)

Adding population table 

In [153]:
new_population_column_names = {
    '0-4_people': 'age_0_4_people',
    '5-9_people': 'age_5_9_people',
    '10-14_people': 'age_10_14_people',
    '15-19_people': 'age_15_19_people',
    '20-24_people': 'age_20_24_people',
    '25-29_people': 'age_25_29_people',
    '30-34_people': 'age_30_34_people',
    '35-39_people': 'age_35_39_people',
    '40-44_people': 'age_40_44_people',
    '45-49_people': 'age_45_49_people',
    '50-54_people': 'age_50_54_people',
    '55-59_people': 'age_55_59_people',
    '60-64_people': 'age_60_64_people',
    '65-69_people': 'age_65_69_people',
    '70-74_people': 'age_70_74_people',
    '75-79_people': 'age_75_79_people',
    '80-84_people': 'age_80_84_people',
    '85-and-over_people': 'age_85_and_over_people',
    'total_people': 'total_people'
}

population.rename(columns=new_population_column_names, inplace=True)
population['total_children'] = population['age_0_4_people'] + population['age_5_9_people'] + population['age_10_14_people'] + population['age_15_19_people']

In [154]:
population

,sa2_code,sa2_name,age_0_4_people,age_5_9_people,age_10_14_people,age_15_19_people,age_20_24_people,age_25_29_people,age_30_34_people,age_35_39_people,...,age_50_54_people,age_55_59_people,age_60_64_people,age_65_69_people,age_70_74_people,age_75_79_people,age_80_84_people,age_85_and_over_people,total_people,total_children
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,602,570,520,464,369,226,142,70,7530,2121
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,794,895,863,925,603,331,264,11052,2471
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,436,422,397,327,264,190,100,75,4748,961
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,882,901,930,917,1065,976,773,1028,14803,3205
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1241,1377,1285,1166,949,664,476,537,21346,4364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,2,4,10,4,9,7,1,2,...,0,0,0,1,0,0,0,0,45,20
369,128021538,Sutherland - Kirrawee,1519,1292,1150,1117,1335,1852,2120,1945,...,1391,1285,1157,909,909,781,582,807,23369,5078
370,128021607,Engadine,1157,1283,1469,1209,891,675,928,1229,...,1086,909,764,707,886,748,389,327,17379,5118
371,128021608,Loftus - Yarrawarrah,503,487,575,508,380,293,426,493,...,477,450,387,418,335,263,192,109,7354,2073


In [155]:
conn.execute("""
DROP TABLE IF EXISTS Population;
CREATE TABLE Population (
    sa2_code VARCHAR(10),
    sa2_name VARCHAR(255),
    age_0_4_people INT,
    age_5_9_people INT,
    age_10_14_people INT,
    age_15_19_people INT,
    age_20_24_people INT,
    age_25_29_people INT,
    age_30_34_people INT,
    age_35_39_people INT,
    age_40_44_people INT,
    age_45_49_people INT,
    age_50_54_people INT,
    age_55_59_people INT,
    age_60_64_people INT,
    age_65_69_people INT,
    age_70_74_people INT,
    age_75_79_people INT,
    age_80_84_people INT,
    age_85_and_over_people INT,
    total_people INT,
    total_children INT, 
    PRIMARY KEY (sa2_code)
);
"""
) 

In [156]:
population.to_sql("population", con=conn, if_exists='append', index=False)
query(conn, "select * from population")

,sa2_code,sa2_name,age_0_4_people,age_5_9_people,age_10_14_people,age_15_19_people,age_20_24_people,age_25_29_people,age_30_34_people,age_35_39_people,...,age_50_54_people,age_55_59_people,age_60_64_people,age_65_69_people,age_70_74_people,age_75_79_people,age_80_84_people,age_85_and_over_people,total_people,total_children
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,602,570,520,464,369,226,142,70,7530,2121
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,794,895,863,925,603,331,264,11052,2471
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,436,422,397,327,264,190,100,75,4748,961
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,882,901,930,917,1065,976,773,1028,14803,3205
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1241,1377,1285,1166,949,664,476,537,21346,4364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,2,4,10,4,9,7,1,2,...,0,0,0,1,0,0,0,0,45,20
369,128021538,Sutherland - Kirrawee,1519,1292,1150,1117,1335,1852,2120,1945,...,1391,1285,1157,909,909,781,582,807,23369,5078
370,128021607,Engadine,1157,1283,1469,1209,891,675,928,1229,...,1086,909,764,707,886,748,389,327,17379,5118
371,128021608,Loftus - Yarrawarrah,503,487,575,508,380,293,426,493,...,477,450,387,418,335,263,192,109,7354,2073


In [157]:
# This is wrong it outputs some really large numbers - MAYBE IT'S THE TOTAL AREA

sql = """
SELECT gf.sa2_code21, gf.sa2_name21, round(1000*ST_Area(gf.geom::geography)/pop.total_children) AS school_area_1000
FROM (gdf gf LEFT JOIN schools s ON ST_Intersects(gf.geom, s.geom)) LEFT JOIN population pop on pop.sa2_code = gf.sa2_code21
WHERE total_children > 0
GROUP BY gf.sa2_code21, gf.sa2_name21, gf.geom, total_children
ORDER BY school_area_1000 DESC;"""
sa2_school_area_per_1000 = query(conn, sql)

# Task 2

I currently can't work with schools at all - don't know what the data is telling me

An approach is to define the SA2 areas by GEOMs and then fit the lat/longs in those sections

In [228]:
column_zscores = zscore(public_transport_stops['stop_count'])
public_transport_stops['stop_count_zscores'] = column_zscores
new_stops_column_names = {
    'sa2_code21': 'sa2_code',
    'sa2_name21': 'sa2_name',
    'stop_count': 'stop_count',
    'stop_count_zscores': 'stop_count_zscores'
}

public_transport_stops.rename(columns=new_stops_column_names, inplace=True)

In [229]:
public_transport_stops

,sa2_code,sa2_name,stop_count,stop_count_zscores
0,115021297,Dural - Kenthurst - Wisemans Ferry,697,6.333921
1,124011454,Springwood - Winmalee,435,3.303733
2,124011452,Katoomba - Leura,395,2.841108
3,102011040,Umina - Booker Bay - Patonga,387,2.748583
4,123021437,Campbelltown - Woodbine,375,2.609796
...,...,...,...,...
368,119041671,Wolli Creek,12,-1.588519
369,127011592,Badgerys Creek,10,-1.611650
370,124011451,Blue Mountains - North,4,-1.681044
371,124021456,Blue Mountains - South,2,-1.704175


In [226]:
column_zscores = zscore(sa2_school_area_per_1000['school_area_1000'])
sa2_school_area_per_1000['school_area_1000_zscores'] = column_zscores
new_schools_column_names = {
    'sa2_code21': 'sa2_code',
    'sa2_name21': 'sa2_name',
    'school_area_1000': 'school_area_1000',
    'school_area_1000_zscores': 'school_area_1000_zscores'
}
sa2_school_area_per_1000.rename(columns=new_schools_column_names, inplace=True)

In [227]:
sa2_school_area_per_1000

,sa2_code,sa2_name,school_area_1000,school_area_1000_zscores
0,127011592,Badgerys Creek,1.757726e+10,16.562866
1,128021537,Royal National Park,6.966690e+09,6.492822
2,127021521,Wetherill Park Industrial,4.278346e+09,3.941427
3,115031299,Bilpin - Colo - St Albans,3.576424e+09,3.275262
4,125011475,Rookwood Cemetery,3.015049e+09,2.742484
...,...,...,...,...
363,119011571,Bankstown - North,6.027160e+05,-0.118400
364,117031641,Pyrmont,5.685130e+05,-0.118433
365,127031731,Liverpool - East,5.531340e+05,-0.118447
366,119021574,Wiley Park,4.770770e+05,-0.118519


## SA2 Busyness Scores

In [199]:
polling

,fid,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,latitude,longitude,geom
13,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,58,1,Oatley,Oatley Public School,51 Letitia St,NaN,NaN,OATLEY,NSW,2223.0,-33.984700,151.081000,POINT (-33.9847 151.081)
15,aec_federal_election_polling_places_2019.fid-4...,NSW,111,Chifley,392,1,Dharruk,Dawson Public School,7 Stuart Rd,NaN,NaN,DHARRUK,NSW,2770.0,-33.747500,150.817000,POINT (-33.7475 150.817)
16,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,31,1,Allawah,PJ Ferry Reserve Community Hall,147B Bellevue Pde,NaN,NaN,ALLAWAH,NSW,2218.0,-33.976790,151.114897,POINT (-33.9767897 151.1148974)
17,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,67,1,Allawah South,St Raphael's Church Hall,84 George St,NaN,NaN,SOUTH HURSTVILLE,NSW,2221.0,-33.975600,151.111000,POINT (-33.9756 151.111)
18,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,56500,1,Beverly Hills North (Banks),Beverly Hills North Public School,1-3 Shorter Ave,NaN,NaN,BEVERLY HILLS,NSW,2209.0,-33.941300,151.075000,POINT (-33.9413 151.075)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2810,1,Warilla North,Warilla North Community Centre,2-6 Hill St,NaN,NaN,WARILLA,NSW,2528.0,-34.547240,150.851770,POINT (-34.54724 150.85177)
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,NaN,NaN,BARRACK HEIGHTS,NSW,2528.0,-34.564200,150.858000,POINT (-34.5642 150.858)
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,NaN,NaN,NaN,WARILLA,NSW,2528.0,-34.550823,150.859755,POINT (-34.5508228 150.8597546)
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,NaN,NaN,WELBY,NSW,2575.0,-34.440900,150.424000,POINT (-34.4409 150.424)


In [200]:
polling['geom'] = gpd.points_from_xy(polling.longitude, polling.latitude)
polling = polling.drop(columns=['latitude', 'longitude']) 

,fid,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,geom
13,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,58,1,Oatley,Oatley Public School,51 Letitia St,NaN,NaN,OATLEY,NSW,2223.0,POINT (151.08100 -33.98470)
15,aec_federal_election_polling_places_2019.fid-4...,NSW,111,Chifley,392,1,Dharruk,Dawson Public School,7 Stuart Rd,NaN,NaN,DHARRUK,NSW,2770.0,POINT (150.81700 -33.74750)
16,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,31,1,Allawah,PJ Ferry Reserve Community Hall,147B Bellevue Pde,NaN,NaN,ALLAWAH,NSW,2218.0,POINT (151.11490 -33.97679)
17,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,67,1,Allawah South,St Raphael's Church Hall,84 George St,NaN,NaN,SOUTH HURSTVILLE,NSW,2221.0,POINT (151.11100 -33.97560)
18,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,56500,1,Beverly Hills North (Banks),Beverly Hills North Public School,1-3 Shorter Ave,NaN,NaN,BEVERLY HILLS,NSW,2209.0,POINT (151.07500 -33.94130)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2810,1,Warilla North,Warilla North Community Centre,2-6 Hill St,NaN,NaN,WARILLA,NSW,2528.0,POINT (150.85177 -34.54724)
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,NaN,NaN,BARRACK HEIGHTS,NSW,2528.0,POINT (150.85800 -34.56420)
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,NaN,NaN,NaN,WARILLA,NSW,2528.0,POINT (150.85975 -34.55082)
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,NaN,NaN,WELBY,NSW,2575.0,POINT (150.42400 -34.44090)


In [202]:
polling['geom'] = polling['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [203]:
#pollingold = polling.copy()  
#polling.columns = map(str.lower, polling.columns)
#polling = polling.dropna(subset=['the_geom'])
#polling['geom'] = polling['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
#polling['geom'] = polling['the_geom']
#polling = polling.drop(columns='the_geom')
#polling

In [204]:
conn.execute("""
DROP TABLE IF EXISTS Polling;
CREATE TABLE Polling (
    fid VARCHAR(255),
    state VARCHAR(255),
    division_id INT,
    division_name VARCHAR(255),
    polling_place_id INT,
    polling_place_type_id INT,
    polling_place_name VARCHAR(255),
    premises_name VARCHAR(255),
    premises_address_1 VARCHAR(255),
    premises_address_2 VARCHAR(255),
    premises_address_3 VARCHAR(255),
    premises_suburb VARCHAR(255),
    premises_state_abbreviation VARCHAR(255),
    premises_post_code FLOAT,
    geom GEOMETRY(POINT, 4283)
);"""
) 

In [205]:
polling.to_sql("polling", con=conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
#query(conn, "select * from polling")

,fid,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,geom
0,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,58,1,Oatley,Oatley Public School,51 Letitia St,None,None,OATLEY,NSW,2223.0,0101000020BB1000003BDF4F8D97E26240832F4CA60AFE...
1,aec_federal_election_polling_places_2019.fid-4...,NSW,111,Chifley,392,1,Dharruk,Dawson Public School,7 Stuart Rd,None,None,DHARRUK,NSW,2770.0,0101000020BB100000A01A2FDD24DA624048E17A14AEDF...
2,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,31,1,Allawah,PJ Ferry Reserve Community Hall,147B Bellevue Pde,None,None,ALLAWAH,NSW,2218.0,0101000020BB100000A7EC4F3DADE36240EA48E47107FD...
3,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,67,1,Allawah South,St Raphael's Church Hall,84 George St,None,None,SOUTH HURSTVILLE,NSW,2221.0,0101000020BB100000643BDF4F8DE3624022FDF675E0FC...
4,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,56500,1,Beverly Hills North (Banks),Beverly Hills North Public School,1-3 Shorter Ave,None,None,BEVERLY HILLS,NSW,2209.0,0101000020BB1000006666666666E26240C6DCB5847CF8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2810,1,Warilla North,Warilla North Community Centre,2-6 Hill St,None,None,WARILLA,NSW,2528.0,0101000020BB100000D8B628B341DB62401288D7F50B46...
2786,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,None,None,BARRACK HEIGHTS,NSW,2528.0,0101000020BB100000FA7E6ABC74DB62409C33A2B43748...
2787,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,None,None,None,WARILLA,NSW,2528.0,0101000020BB100000BD32141C83DB624011F28B5C8146...
2788,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,None,None,WELBY,NSW,2575.0,0101000020BB10000021B0726891CD6240386744696F38...


In [214]:
sql = """
SELECT gf.sa2_code21, gf.sa2_name21, COUNT(pol.premises_name) AS polling_count
FROM gdf gf LEFT JOIN polling pol ON ST_Intersects(pol.geom, gf.geom)
GROUP BY gf.sa2_code21, gf.sa2_name21
ORDER BY polling_count DESC;
"""
polling_count = query(conn, sql)

In [224]:
column_zscores = zscore(polling_count['polling_count'])
polling_count['polling_count_zscores'] = column_zscores
new_polling_column_names = {
    'sa2_code21': 'sa2_code',
    'sa2_name21': 'sa2_name',
    'polling_count': 'polling_count',
    'polling_count_zscores': 'polling_count_zscores'
}

polling_count.rename(columns=new_polling_column_names, inplace=True)

In [225]:
polling_count

,sa2_code,sa2_name,polling_count,polling_count_zscores
0,117031644,Sydney (North) - Millers Point,64,14.540750
1,117031645,Sydney (South) - Haymarket,27,5.563525
2,125041717,Parramatta - North,18,3.379876
3,121011684,Chatswood - East,16,2.894620
4,124031464,Penrith,13,2.166737
...,...,...,...,...
368,128021537,Royal National Park,0,-0.987423
369,124021456,Blue Mountains - South,0,-0.987423
370,115041623,Box Hill - Nelson,0,-0.987423
371,116021629,Marsden Park - Shanes Park,0,-0.987423


# Businesses Data

In [207]:
businesses

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...
12212,S,Other Services,128021538,Sutherland - Kirrawee,21,66,58,3,3,0,152
12213,S,Other Services,128021607,Engadine,13,41,31,3,0,0,87
12214,S,Other Services,128021608,Loftus - Yarrawarrah,0,10,10,0,0,0,22
12215,S,Other Services,128021609,Woronora Heights,0,3,5,0,0,0,9


In [ ]:
conn.execute("""
DROP TABLE IF EXISTS Business;
CREATE TABLE Business (
    industry_code VARCHAR(255),
    industry_name VARCHAR(255),
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    businesses_to_0_50k INT,
    businesses_50k_to_200k INT,
    businesses_200k_to_2m INT,
    businesses_2m_to_5m INT,
    businesses_5m_to_10m INT,
    businesses_10m_or_more INT,
    total_businesses INT
);"""
) 

In [208]:
businesses.to_sql("business", con=conn, if_exists='append', index=False)
query(conn, "select * from business")

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...
12212,S,Other Services,128021538,Sutherland - Kirrawee,21,66,58,3,3,0,152
12213,S,Other Services,128021607,Engadine,13,41,31,3,0,0,87
12214,S,Other Services,128021608,Loftus - Yarrawarrah,0,10,10,0,0,0,22
12215,S,Other Services,128021609,Woronora Heights,0,3,5,0,0,0,9


In [219]:
sql = """
SELECT sa2_code, sa2_name, SUM(total_businesses) AS total_businesses
FROM business
GROUP BY sa2_code, sa2_name
ORDER BY total_businesses DESC;
"""
businesses_count = query(conn, sql)

In [ ]:
# column_zscores = zscore(businesses_count['total_businesses'])
businesses_count['total_businesses_zscores'] = column_zscores

In [222]:
businesses_count

,sa2_code,sa2_name,total_businesses,total_businesses_zscores
0,117031644,Sydney (North) - Millers Point,37597.0,21.544384
1,117031645,Sydney (South) - Haymarket,8333.0,4.155205
2,199999499,Currently Unknown,7457.0,3.634670
3,121041417,North Sydney - Lavender Bay,6652.0,3.156325
4,115011291,Baulkham Hills (West) - Bella Vista,5808.0,2.654805
...,...,...,...,...
638,107021135,Illawarra Catchment Reserve,0.0,-0.796409
639,124021456,Blue Mountains - South,0.0,-0.796409
640,123021439,Holsworthy Military Area,0.0,-0.796409
641,118011342,Centennial Park,0.0,-0.796409


# Calculation of Z Score

In [238]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

businesses_count['sa2_code'] = businesses_count['sa2_code'].astype(int)
polling_count['sa2_code'] = polling_count['sa2_code'].astype(int)
sa2_school_area_per_1000['sa2_code'] = sa2_school_area_per_1000['sa2_code'].astype(int)
public_transport_stops['sa2_code'] = public_transport_stops['sa2_code'].astype(int)

merged_df = pd.merge(businesses_count, polling_count, on='sa2_code', suffixes=('businesses', 'polling'), how='inner')
merged_df = pd.merge(merged_df, sa2_school_area_per_1000, on='sa2_code', suffixes=('merged', 'school'), how='inner')
merged_df = pd.merge(merged_df, public_transport_stops, on='sa2_code', suffixes=('merged', 'public_transport'), how='inner')

merged_df.fillna(0, inplace=True)

merged_df['sum_z_scores'] = merged_df[['total_businesses_zscores', 'polling_count_zscores', 'school_area_1000_zscores', 'stop_count_zscores']].sum(axis=1)
merged_df['sigmoid_value'] = sigmoid(merged_df['sum_z_scores'])

print(merged_df[['sa2_code', 'sa2_namebusinesses', 'sigmoid_value']])

      sa2_code                   sa2_namebusinesses  sigmoid_value
0    117031644       Sydney (North) - Millers Point       1.000000
1    117031645           Sydney (South) - Haymarket       0.999844
2    121041417          North Sydney - Lavender Bay       0.936653
3    115011291  Baulkham Hills (West) - Bella Vista       0.991111
4    121011684                     Chatswood - East       0.994306
..         ...                                  ...            ...
363  117011324               Port Botany Industrial       0.339035
364  117011325                       Sydney Airport       0.113144
365  125011475                    Rookwood Cemetery       0.414123
366  127011592                       Badgerys Creek       0.999998
367  128021537                  Royal National Park       0.965070

[368 rows x 3 columns]


# Task 3
Extend the score by sourcing one additional dataset for each group member, and then incorporating all new datasets into your scoring function. For full marks, at least one dataset should be of spatial data, and at least one should be of a type not used so far in this assignment (e.g. JSON, XML, or collated via web scraping). Almost any subject matter is permissible, so long as it can be justified as relevant to the calculation of our ”bustling” metric (e.g. public facilities, other census statistics, local wildlife, etc).For either version of your scoring function (or both!), the following subtasks should also be achieved:


1. Visualise your score in an engaging way, and summarise key results in a table (ideally including a useful map-overlay visualisation, or an interactive graph).
2. Include in-depth analysis into your results. Note interesting findings, discuss their limitations, and summarise key conclusions.
3. Determine if there is any correlation between your score and the median income of each region.
4. Ensure at least one useful index (ideally spatial) has been used for your calculations